In [ ]:
import pandas as pd
import datetime
pd.set_option('display.max_colwidth', 128)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 60)

In [ ]:
"""
-- Modified original query from https://dune.com/queries/92408/184718

SELECT 
  tx.hash,
  tx.success,
  --pid."name", 
  mints."_projectId" AS ProjectID,
  tx.value/1e18 AS price_eth,
  date_trunc('second', mints."evt_block_time") AS time,   
  mints."_to" AS buyer, 
  (tx."gas_used" * tx."gas_price"/1e18) AS gas_eth
FROM artblocks."GenArt721_evt_Mint" mints -- old contrct
LEFT JOIN ethereum.transactions tx
  ON mints."evt_tx_hash" = tx."hash"
--LEFT JOIN dune_user_generated.ArtBlocksProjectIDs pid 
--  ON pid.id = mints."_projectId"

UNION ALL 
    
SELECT 
  tx.hash,
  tx.success,
  --pid."name", 
  mints."_projectId" AS ProjectID, 
  tx.value/1e18 AS price, 
  date_trunc('second', mints."call_block_time") AS time, 
  mints."_by" AS buyer, 
  (tx."gas_used" * tx."gas_price"/1e18) AS gas_eth
FROM artblocks."GenArt721Core_call_mint" mints -- new contract
LEFT JOIN ethereum.transactions tx
  ON mints."call_tx_hash" = tx."hash"
--LEFT JOIN dune_user_generated.ArtBlocksProjectIDs pid 
--  ON pid.id = mints."_projectId"
WHERE "output__tokenId" is not null
ORDER BY time DESC
"""

d = pd.read_csv('/Users/davidma/Downloads/0fb786aa-901e-4f31-85ae-65df8613e50c.csv')
d["time"] = pd.to_datetime(d["time"])
display(d.dtypes)

# sort by time and descending gas for most probable execution order without looking at transaction order numbers.
d.sort_values(by=["time", "gas_eth"], ascending=[True, False], inplace=True)

d.head()

In [ ]:
display(d.info())
display("Number of successful mints: ", d.success.sum())
d.describe(include=['bool','float', 'int', 'datetime'])

In [ ]:
mints = d[d["success"]]

import statistics

def getMiddleValue(pdSeries):
    mid = (pdSeries.count() / 2).astype(int)
    return pdSeries.iloc[mid]

mintsByProjectId = mints.groupby("projectid")
mintsByProjectId = pd.DataFrame({
    "count": mintsByProjectId["projectid"].count(),
    "firstMintTime": mintsByProjectId["time"].first(),
    "lastMintTime": mintsByProjectId["time"].last(),
    "lastMintPriceTotal": mintsByProjectId["price_eth"].last() + mintsByProjectId["gas_eth"].last(),
    "medianMintTime":  mintsByProjectId["time"].apply(lambda x: getMiddleValue(x))
    })
mintsByProjectId["latterMintWindowInMins"] = round((mintsByProjectId["lastMintTime"] - mintsByProjectId["medianMintTime"]).dt.total_seconds() / 60, 2)


In [ ]:
"""
-- Modified original query from https://dune.com/queries/160701/314169

select distinct block_time, 
  ROUND("nft_token_id"::numeric / 1000000) as projectid,
  round(eth_amount, 2) as eth_price, 
  usd_price, 
  link, 
  platform, 
  left(seller::text, 7) as seller, 
  left(buyer::text, 7) as buyer 
from 
(
select 
  block_time, 
  platform, 
  usd_amount, 
     
  case 
     when ("original_currency" = 'ETH' OR "original_currency" = 'WETH')
             THEN  ("original_amount")
    else 0  
  END as eth_amount, 
  "usd_amount" as usd_price,

   
 CONCAT('<a href="https://opensea.io/assets/', CONCAT('0x', substring(a."nft_contract_address"::text from 3)), '/', a.nft_token_id,  '/?ref=0x8F903cFC0Af3C2EC0d872c57538AF5e071544a57','" target="_blank" >', 'View on OS', '</a>') as  link,
   
 seller, 
 buyer, 
 tx_hash,
 nft_token_id

from nft."trades" a
WHERE 
     "trade_type" = 'Single Item Trade'
     AND (a.nft_contract_address = '\xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270'
    OR  a.nft_contract_address = '\x059edd72cd353df5106d2b9cc5ab83a52287ac3a')
ORDER BY block_time DESC 
) gg
-- WHERE block_time > '{{Date}}'
order by block_time DESC
"""

p = pd.read_csv('/Users/davidma/Downloads/9c238c88-a15b-4479-a8a2-c78f4a8f1385.csv')

display("Before filtering:", len(d))

# cleaning up weird project ids
p_removed = p[p["projectid"].str.len() >= 8]
p = p[p["projectid"].str.len() < 8]

# casting
p["block_time"] = pd.to_datetime(p["block_time"])
p["projectid"] = p["projectid"].astype(int)

p.sort_values(by=["block_time"], ascending=[True], inplace=True)


# adding derived data
p["normalized_price"] = p["eth_price"] / p["projectid"].map(mintsByProjectId.lastMintPriceTotal)
p["lastMintTime"] = p["projectid"].map(mintsByProjectId.lastMintTime)
p["isWithin2hFromLastMintTime"] = ((p["block_time"] - p["lastMintTime"]).dt.total_seconds() / 60 < 120) & ((p["block_time"] - p["lastMintTime"]).dt.total_seconds() > 0)

display(p.dtypes)
display(p.describe(include=['bool','float', 'int', 'datetime']))
p

In [ ]:
tradesByProjectId = p.groupby("projectid")
tradesByProjectId = pd.DataFrame({
    "tradeCount": tradesByProjectId["projectid"].count(),
    "tradeCount2hr": tradesByProjectId["isWithin2hFromLastMintTime"].sum(),
    "medianNormPrice2h": tradesByProjectId.apply(lambda df: df[df["isWithin2hFromLastMintTime"]].normalized_price.median()),
})

In [ ]:
byProjectId = pd.merge(mintsByProjectId, tradesByProjectId, on="projectid", how="outer")

In [ ]:
from ipywidgets import *

def update(projectid = 331):
    p334 = p[p["projectid"] == projectid]
    p334[p334["isWithin2hFromLastMintTime"]].plot(x="block_time", y=["normalized_price", "eth_price"], figsize=(20,8))

interact(update)

In [ ]:
view = mints[mints["projectid"] == 337]